In [7]:
# Level-31 — Adaptive event labeling + calibrated GradientBoosting + backtest (hardened 1-D handling)

import warnings
warnings.filterwarnings("ignore")

import math
import inspect
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta, timezone

from sklearn.model_selection import TimeSeriesSplit
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

# ----------------------------- Config -----------------------------
TICKER       = "AAPL"
YEARS        = 3.0          # years of daily data
FREQ         = "1D"         # daily bars
VOL_SPAN     = 50           # EWMA vol span
H_BARS       = 10           # vertical barrier in bars
CUSUM_GRID   = [0.002, 0.003, 0.004, 0.006, 0.008, 0.010]  # thresholds to try
UP_M, DN_M   = 3.0, 3.0     # triple-barrier multipliers (in vols)
MIN_EVENTS   = 500          # ensure enough labels
KFOLDS       = 5
RNG_SEED     = 42
TC_BP        = 5            # transaction cost (bp) per toggle
MAX_DAILY_TURNOVER = 0.5    # cap for daily turnover

np.random.seed(RNG_SEED)

# ----------------------------- Utils -----------------------------
def utc_now_date():
    return datetime.now(timezone.utc).date()

def load_prices(ticker, years, freq="1D"):
    # Pulls more days than needed to be safe
    end = utc_now_date()
    start = (datetime.now(timezone.utc) - timedelta(days=int(365*years + 20))).date()
    df = yf.download(ticker, start=start, end=end, auto_adjust=True, progress=False)
    if df.empty:
        raise SystemExit("No data downloaded.")
    if "Adj Close" in df.columns:
        s = df["Adj Close"].copy()
    elif "Close" in df.columns:
        s = df["Close"].copy()
    else:
        raise RuntimeError("No Close/Adj Close in downloaded data.")
    s.name = "Close"
    s = s.asfreq("B").ffill()   # business days, forward-fill
    return s

def ewma_vol(r, span=50):
    return r.ewm(span=span, adjust=False).std()

def to_1d_series(r, index=None) -> pd.Series:
    """
    Normalize r to a 1-D float Series with a proper index.
    Accepts Series, DataFrame (uses first column), or any ndarray-like; squeezes to 1-D.
    """
    if isinstance(r, pd.Series):
        s = r.copy()
        s = s.astype("float64")
        return s.fillna(0.0)

    if isinstance(r, pd.DataFrame):
        # use first column
        s = r.iloc[:, 0].copy()
        s = s.astype("float64")
        return s.fillna(0.0)

    # ndarray-like
    arr = np.asarray(r)
    arr = np.ravel(arr)  # squeeze to 1-D safely
    if index is None:
        index = pd.RangeIndex(len(arr))
    return pd.Series(arr, index=index, dtype="float64").fillna(0.0)

def cusum_filter(r, threshold, index=None) -> pd.DatetimeIndex:
    """
    Symmetric CUSUM robust to Series/DataFrame/ndarray inputs.
    Always converts to 1-D float Series (keeping provided index if given),
    then iterates over plain floats.
    """
    s = to_1d_series(r, index=index)
    idx = s.index
    vals = s.to_numpy(dtype=float)

    s_pos, s_neg = 0.0, 0.0
    t_events = []
    for i, x in enumerate(vals):
        x = float(x)
        s_pos = max(0.0, s_pos + x)
        s_neg = min(0.0, s_neg + x)
        if s_pos > threshold:
            s_pos = 0.0
            t_events.append(idx[i])
        elif s_neg < -threshold:
            s_neg = 0.0
            t_events.append(idx[i])
    return pd.DatetimeIndex(t_events).unique().sort_values()

def get_vertical_barriers(t_events: pd.DatetimeIndex, h: int, index: pd.DatetimeIndex) -> pd.Series:
    if len(t_events) == 0:
        return pd.Series(dtype="datetime64[ns]")
    out = {}
    for t0 in t_events:
        pos = index.get_indexer([t0])[0]
        t1_pos = min(pos + h, len(index) - 1)
        out[t0] = index[t1_pos]
    return pd.Series(out)

def _first_true_index(bool_series: pd.Series):
    """Return the index label of the first True in a boolean Series, or None if none."""
    if bool_series is None or len(bool_series) == 0:
        return None
    # Ensure we only call argmax when there's at least one True
    if not bool(bool_series.any()):
        return None
    arr = bool_series.to_numpy(dtype=bool)
    i = int(np.argmax(arr))
    return bool_series.index[i]

def get_triple_barrier_labels(close, t_events, vbar, up_m, dn_m, daily_vol):
    # Align and build target
    trgt = daily_vol.reindex(t_events).bfill().ffill()
    df = pd.DataFrame({"t1": vbar.reindex(t_events), "trgt": trgt}, index=t_events).dropna(subset=["t1", "trgt"])
    labels = []
    for t0, row in df.iterrows():
        t1 = pd.Timestamp(row["t1"])
        c0 = float(close.loc[t0])
        up_lvl = c0 * (1 + up_m * float(row["trgt"]))
        dn_lvl = c0 * (1 - dn_m * float(row["trgt"]))
        # slice path (inclusive)
        path = to_1d_series(close.loc[t0:t1])
        if path.empty:
            continue
        path_up = (path >= up_lvl)
        path_dn = (path <= dn_lvl)

        hit_up = _first_true_index(path_up)
        hit_dn = _first_true_index(path_dn)

        if (hit_up is not None) and (hit_dn is not None):
            lbl = 1 if hit_up <= hit_dn else 0
            t_end = hit_up if lbl == 1 else hit_dn
        elif hit_up is not None:
            lbl, t_end = 1, hit_up
        elif hit_dn is not None:
            lbl, t_end = 0, hit_dn
        else:
            c1 = float(close.loc[t1])
            lbl, t_end = (1 if c1 > c0 else 0), t1

        labels.append((t0, pd.Timestamp(t_end), int(lbl), float(row["trgt"])))

    if not labels:
        return pd.DataFrame(columns=["t1", "label", "trgt"])
    out = pd.DataFrame(labels, columns=["t0", "t1", "label", "trgt"]).set_index("t0")
    return out

def make_features(close: pd.Series) -> pd.DataFrame:
    r = close.pct_change().replace([np.inf, -np.inf], 0.0).fillna(0.0)
    f = pd.DataFrame(index=close.index)
    f["r1"] = r
    f["r5"] = close.pct_change(5)
    f["r10"] = close.pct_change(10)
    f["mom5"] = (close / close.shift(5) - 1.0)
    f["mom10"] = (close / close.shift(10) - 1.0)
    f["vol10"] = r.rolling(10).std()
    f["vol20"] = r.rolling(20).std()
    f["z20"] = (close - close.rolling(20).mean()) / (1e-12 + close.rolling(20).std())
    delta = close.diff()
    up = delta.clip(lower=0.0).rolling(14).mean()
    dn = (-delta.clip(upper=0.0)).rolling(14).mean()
    rs = up / (1e-12 + dn)
    f["rsi14"] = 100.0 - 100.0 / (1.0 + rs)
    return f.replace([np.inf, -np.inf], 0.0).fillna(0.0)

def sample_weights_from_trgt(trgt: pd.Series) -> pd.Series:
    w = 1.0 / (1e-8 + trgt)
    return w.clip(upper=np.quantile(w, 0.99))

def choose_proba_threshold(y_true: pd.Series, proba: pd.Series) -> float:
    grid = np.linspace(0.4, 0.7, 16)
    best_t, best_s = 0.5, -1e9
    for t in grid:
        pred = (proba >= t).astype(int)
        if int(pred.sum()) == 0:
            score = -1e9
        else:
            tp = ((pred == 1) & (y_true == 1)).mean()
            fp = ((pred == 1) & (y_true == 0)).mean()
            score = float(tp - 0.5 * fp)
        if score > best_s:
            best_s, best_t = score, t
    return float(best_t)

def sharpe_ratio(x: pd.Series) -> float:
    std = x.std()
    if std == 0 or np.isnan(std):
        return 0.0
    return float(np.sqrt(252) * x.mean() / std)

def drawdown(x: pd.Series) -> float:
    cum = (1 + x).cumprod()
    peak = cum.cummax()
    dd = (cum / peak - 1.0).min()
    return float(dd)

def make_calibrator(base, method="isotonic", cv="prefit"):
    sig = inspect.signature(CalibratedClassifierCV)
    if "estimator" in sig.parameters:
        return CalibratedClassifierCV(estimator=base, method=method, cv=cv)
    else:
        return CalibratedClassifierCV(base_estimator=base, method=method, cv=cv)

def adaptive_events_and_labels(close, rets, base_H, cusum_grid, vol_span, up_m, dn_m, min_events):
    daily_vol = ewma_vol(to_1d_series(rets, index=close.index), span=vol_span).clip(lower=1e-8).fillna(0.0)
    used_thr = None
    events_idx = pd.DatetimeIndex([])
    for thr in cusum_grid:
        ev = cusum_filter(rets, thr, index=close.index)  # pass explicit index
        if len(ev) >= min_events:
            used_thr = thr
            events_idx = ev
            break
    if used_thr is None:
        used_thr = min(cusum_grid)
        events_idx = cusum_filter(rets, used_thr, index=close.index)

    vbar = get_vertical_barriers(events_idx, base_H, close.index)
    labels = get_triple_barrier_labels(close, events_idx, vbar, up_m, dn_m, daily_vol)
    labels = labels.dropna()
    if labels.empty:
        raise SystemExit("No labeled events even after trying CUSUM thresholds. Adjust CUSUM_GRID/UP_M/DN_M/VOL_SPAN.")
    print(f"[Adaptive] events={len(labels)}  H={base_H}  thr≈{used_thr}")
    return labels, daily_vol, events_idx, base_H, float(used_thr)

# ----------------------------- Main -----------------------------
if __name__ == "__main__":
    # 1) Data
    close = load_prices(TICKER, YEARS, FREQ)
    rets  = close.pct_change().replace([np.inf, -np.inf], 0.0).fillna(0.0)  # Series, but we guard anyway downstream

    # 2) Events & Labels (adaptive CUSUM)
    labels, daily_vol, events_idx, H_used, thr_used = adaptive_events_and_labels(
        close, rets, H_BARS, CUSUM_GRID, VOL_SPAN, UP_M, DN_M, MIN_EVENTS
    )

    # 3) Features aligned to event times
    feats = make_features(close)
    X = feats.reindex(labels.index).dropna()
    y = labels.loc[X.index, "label"].astype(int)
    w = sample_weights_from_trgt(labels.loc[X.index, "trgt"])

    # 4) Time-series CV with calibration and threshold selection
    tscv = TimeSeriesSplit(n_splits=KFOLDS)
    oof_proba = pd.Series(index=X.index, dtype=float)
    oof_pred  = pd.Series(index=X.index, dtype=int)
    thrs = []

    for fold, (tr_idx, va_idx) in enumerate(tscv.split(X), 1):
        tr_ind = X.index[tr_idx]
        va_ind = X.index[va_idx]
        X_tr, y_tr, w_tr = X.loc[tr_ind], y.loc[tr_ind], w.loc[tr_ind]
        X_va, y_va, w_va = X.loc[va_ind], y.loc[va_ind], w.loc[va_ind]

        base = GradientBoostingClassifier(random_state=RNG_SEED)
        base.fit(X_tr, y_tr, sample_weight=w_tr)

        cal = make_calibrator(base, method="isotonic", cv="prefit")
        # Fit calibrator on validation slice
        cal.fit(X_va, y_va, sample_weight=w_va)

        proba_va = pd.Series(cal.predict_proba(X_va)[:, 1], index=va_ind)
        oof_proba.loc[va_ind] = proba_va

        thr_f = choose_proba_threshold(y_va, proba_va)
        thrs.append(thr_f)
        oof_pred.loc[va_ind] = (proba_va >= thr_f).astype(int)

        try:
            auc = roc_auc_score(y_va, proba_va)
        except Exception:
            auc = float("nan")
        print(f"Fold {fold}: AUC={auc:.3f}  thr={thr_f:.2f}")

    chosen_thr = float(np.median(thrs)) if thrs else 0.5
    print(f"\nChosen probability threshold (median over folds): {chosen_thr:.2f}")

    # 5) Final fit on all data (refit + calibrate on last fold split)
    tr_idx, va_idx = list(TimeSeriesSplit(n_splits=KFOLDS).split(X))[-1]
    tr_ind = X.index[tr_idx]
    va_ind = X.index[va_idx]
    base = GradientBoostingClassifier(random_state=RNG_SEED)
    base.fit(X.loc[tr_ind], y.loc[tr_ind], sample_weight=w.loc[tr_ind])
    cal = make_calibrator(base, method="isotonic", cv="prefit")
    cal.fit(X.loc[va_ind], y.loc[va_ind], sample_weight=w.loc[va_ind])

    # 6) Backtest (long/flat on next-day return)
    proba_all = pd.Series(cal.predict_proba(X)[:, 1], index=X.index)
    signal = (proba_all >= chosen_thr).astype(int)

    next_ret = close.pct_change().shift(-1)
    ret_e = next_ret.reindex(signal.index)
    pnl_gross = (signal * ret_e).fillna(0.0)

    toggle = (signal != signal.shift(1)).fillna(False).astype(int)
    avg_turn = float(toggle.mean())
    if avg_turn > MAX_DAILY_TURNOVER and avg_turn > 0:
        scale = MAX_DAILY_TURNOVER / avg_turn
        # Keep signals numeric (0 or scaled 1). Turnover pattern is unchanged.
        signal = (signal * scale)
        toggle = (signal != signal.shift(1)).fillna(False).astype(int)
        print(f"Scaled signals to respect turnover cap. New avg daily turnover ≈ {float(toggle.mean()):.3f}")

    tc = toggle * (TC_BP / 1e4)
    pnl_net = pnl_gross - tc

    # Metrics
    def sharpe_ratio(x: pd.Series) -> float:
        std = x.std()
        if std == 0 or np.isnan(std):
            return 0.0
        return float(np.sqrt(252) * x.mean() / std)

    def drawdown(x: pd.Series) -> float:
        cum = (1 + x).cumprod()
        peak = cum.cummax()
        dd = (cum / peak - 1.0).min()
        return float(dd)

    sharpe = sharpe_ratio(pnl_net)
    cagr_cum = (1 + pnl_net).cumprod()
    if len(cagr_cum) > 1:
        years_span = max(1e-9, (cagr_cum.index[-1] - cagr_cum.index[0]).days / 365.25)
        cagr_val = float(cagr_cum.iloc[-1] ** (1 / years_span) - 1.0) if years_span > 0 else 0.0
    else:
        cagr_val = 0.0
    maxdd = drawdown(pnl_net)
    # Hit rate only when signal > 0 (post-threshold)
    trade_mask = (signal > 0)
    hit = float(((trade_mask) & (ret_e > 0)).sum() / max(1, trade_mask.sum()))

    print("\n=== Walk-forward Backtest (long/flat meta-signal) ===")
    print(f"Sharpe: {sharpe:.3f}  CAGR: {100*cagr_val:.2f}%  MaxDD: {100*maxdd:.2f}%  Hit%: {100*hit:.2f}%")
    print(f"Avg daily turnover: {float(toggle.mean()):.3f}  (Max cap {MAX_DAILY_TURNOVER})")
    print(f"Avg daily TC paid (bp): {float((tc*1e4).mean()):.2f}")

    # 7) Save results
    cv_df = pd.DataFrame({
        "proba": oof_proba,
        "pred":  oof_pred,
        "label": y.reindex(oof_proba.index)
    }).dropna()
    bt_df = pd.DataFrame({
        "signal": signal,
        "next_ret": ret_e,
        "pnl_gross": pnl_gross,
        "toggle": toggle,
        "tc": tc,
        "pnl_net": pnl_net
    }).dropna()

    cv_name = f"{TICKER}_level31_cv_summary.csv"
    bt_name = f"{TICKER}_level31_backtest.csv"
    cv_df.to_csv(cv_name)
    bt_df.to_csv(bt_name)
    print(f"\nSaved: {cv_name}, {bt_name}")


[Adaptive] events=674  H=10  thr≈0.002
Fold 1: AUC=0.606  thr=0.40
Fold 2: AUC=0.544  thr=0.40
Fold 3: AUC=0.537  thr=0.40
Fold 4: AUC=0.554  thr=0.40
Fold 5: AUC=0.568  thr=0.40

Chosen probability threshold (median over folds): 0.40


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().